# 항공 사진 내 선인장 식별 경진대회 탐색적 데이터 분석
- [항공 사진 내 선인장 식별 경진대회 링크](https://www.kaggle.com/c/aerial-cactus-identification)

In [1]:
import pandas as pd

import matplotlib.pyplot as plt

# 데이터 경로
data_path = '/kaggle/input/aerial-cactus-identification/'

labels = pd.read_csv(data_path + 'train.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [2]:
from zipfile import ZipFile

# 훈련 이미지 데이터 압출 풀기
with ZipFile(data_path + 'train.zip') as zipper:
    zipper.extractall()
    
# 테스트 이미지 데이터 압출 풀기
with ZipFile(data_path + 'test.zip') as zipper:
    zipper.extractall()

In [3]:
import os

num_train = len(os.listdir('train/'))
num_test = len(os.listdir('test/'))

print(f'훈련 데이터 개수: {num_train}')
print(f'테스트 데이터 개수: {num_test}')

훈련 데이터 개수: 17500
테스트 데이터 개수: 4000


## Improved Model

### 1. seed 고정

In [4]:
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 2. 데이터 준비

In [6]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(labels,
                               test_size = 0.1,
                               stratify = labels["has_cactus"],
                               random_state = 50)

print(f"훈련 데이터 : {train.shape}")
print(f"검증 데이터 : {valid.shape}")

훈련 데이터 : (15750, 2)
검증 데이터 : (1750, 2)


### 3. 데이터셋 클래스 정의

In [7]:
import cv2
from torch.utils.data import Dataset

In [8]:
class ImageDataset(Dataset):
    def __init__(self, df, img_dir='./', transform=None):
        super().__init__()
        self.df = df
        self.img_dir = img_dir
        self.transform = transform
        
    def __len__(self): # len(ImageDataset)을 호출했을 때 반환값 정의
        return len(self.df)
    
    def __getitem__(self, idx): # ImageDataset[idx] 형식으로 줬을 때 반환 값 정의
        img_id = self.df.iloc[idx, 0]
        img_path = self.img_dir + img_id
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.df.iloc[idx, 1]
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [9]:
from torchvision import transforms

transform_train = transforms.Compose([transforms.ToTensor(),
                                     transforms.Pad(32, padding_mode='symmetric'),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.RandomVerticalFlip(),
                                     transforms.RandomRotation(10),
                                     transforms.Normalize((0.485, 0.456, 0.406),
                                                         (0.229, 0.224, 0.225))])

transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Pad(32, padding_mode='symmetric'),
                                     transforms.Normalize((0.485, 0.456, 0.406),
                                                         (0.229, 0.224, 0.225))])

In [10]:
dataset_train = ImageDataset(train, img_dir = 'train/', transform = transform_train)
dataset_valid = ImageDataset(valid, img_dir = 'train/', transform = transform_test)

### 4. 데이터 로더 생성
- 데이터 로더 : 지정한 배치 크기 만큼 데이터를 불러오는 객체
- 딥러닝 훈련할 때, 주로 배치 단위로 데이터를 불러와 훈련을 하기 때문에 사용됩니다.

In [11]:
from torch.utils.data import DataLoader

loader_train = DataLoader(dataset = dataset_train, batch_size = 32, shuffle=True)
loader_valid = DataLoader(dataset = dataset_valid, batch_size = 32, shuffle=False)

### 5. 모델 생성

In [12]:
import torch.nn as nn # 신경망 모듈
import torch.nn.functional as F # 신경망 모듈에서 자주 사용되는 함수

In [13]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=2),
                                   nn.BatchNorm2d(32),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        self.layer2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=2),
                                   nn.BatchNorm2d(64),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        self.layer3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=2),
                                   nn.BatchNorm2d(128),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        self.layer4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=2),
                                   nn.BatchNorm2d(256),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        self.layer5 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=2),
                                   nn.BatchNorm2d(512),
                                   nn.LeakyReLU(),
                                   nn.MaxPool2d(kernel_size=2))
        
        self.avg_pool = nn.AvgPool2d(kernel_size=4)
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(in_features=512 * 1 * 1, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=2)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.avg_pool(x)
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [14]:
model = Model().to(device) # GPU에 모델을 할당

### 6.손실함수, 옵티마이저 설정 

In [15]:
criterion = nn.CrossEntropyLoss()

In [16]:
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00006) # batch 사이즈가 작으면 학습률도 작게 해줘야한다.

### 7. 모델 훈련

In [17]:
epochs = 70

for epoch in range(epochs):
    epoch_loss = 0
    
    for images, labels in loader_train:
    
        # 이미지, 레이블 데이터 미니배치를 장비에 할당
        images = images.to(device)
        labels = labels.to(device)

        # 옵티마이저 기울기 초기화
        optimizer.zero_grad()

        # 순전파
        outputs = model(images)

        # 손실함수를 활용해 output과 label의 손실값 계산
        loss = criterion(outputs, labels)

        # 현재 배치에서의 손실 추가
        epoch_loss += loss.item()
        loss.backward() # 역전파 수행

        optimizer.step() # 가중치 갱신
        
    print(f"epochs [{epoch+1}/{epochs}] - loss: {epoch_loss/len(loader_train):.4f}")

epochs [1/70] - loss: 0.1305
epochs [2/70] - loss: 0.0704
epochs [3/70] - loss: 0.0515
epochs [4/70] - loss: 0.0420
epochs [5/70] - loss: 0.0399
epochs [6/70] - loss: 0.0354
epochs [7/70] - loss: 0.0293
epochs [8/70] - loss: 0.0296
epochs [9/70] - loss: 0.0259
epochs [10/70] - loss: 0.0242
epochs [11/70] - loss: 0.0232
epochs [12/70] - loss: 0.0226
epochs [13/70] - loss: 0.0222
epochs [14/70] - loss: 0.0214
epochs [15/70] - loss: 0.0202
epochs [16/70] - loss: 0.0210
epochs [17/70] - loss: 0.0176
epochs [18/70] - loss: 0.0183
epochs [19/70] - loss: 0.0170
epochs [20/70] - loss: 0.0156
epochs [21/70] - loss: 0.0158
epochs [22/70] - loss: 0.0148
epochs [23/70] - loss: 0.0132
epochs [24/70] - loss: 0.0139
epochs [25/70] - loss: 0.0133
epochs [26/70] - loss: 0.0105
epochs [27/70] - loss: 0.0137
epochs [28/70] - loss: 0.0114
epochs [29/70] - loss: 0.0085
epochs [30/70] - loss: 0.0094
epochs [31/70] - loss: 0.0104
epochs [32/70] - loss: 0.0090
epochs [33/70] - loss: 0.0088
epochs [34/70] - lo

### 8. 모델 검증

In [18]:
from sklearn.metrics import roc_auc_score

model.eval() # 모델을 평가 상태로 설정

true_list = []
preds_list = []

with torch.no_grad(): # 기울기 계산 비활성화
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds = torch.softmax(outputs.cpu(), dim=1)[:, 1]
        true = labels.cpu()
        
        preds_list.extend(preds)
        true_list.extend(true)
        
print(f"검증 데이터 ROC, AUC : {roc_auc_score(true_list, preds_list):.4f}")

검증 데이터 ROC, AUC : 0.9998


### 9. 예측 및 결과 제출

In [19]:
dataset_test = ImageDataset(df = submission, img_dir = "test/", transform = transform_test)
loader_test = DataLoader(dataset = dataset_test, batch_size = 32, shuffle = False)

In [20]:
model.eval()

preds = []

with torch.no_grad():
    for images, _ in loader_test:
        images = images.to(device)
        
        outputs = model(images)
        
        preds_part = torch.softmax(outputs.cpu(), dim=1)[:, 1].tolist() # 텐서는 제출할 수 없기에, 1차원 텐서를 리스트로 변환해줘야됨
        preds.extend(preds_part)

In [21]:
submission['has_cactus'] = preds
submission.to_csv('submission.csv', index=False)

In [22]:
import shutil

# 경로의 디렉토리 전체를 삭제하는 함수
shutil.rmtree('./train')
shutil.rmtree('./test')